# REVIEWS

## REVIEWS 1 - Remove the commas and leave only 3 columns

In [ ]:
# Read the CSV file with the correct separator
df = pd.read_csv('data/movie_reviews1.csv', sep=',', skipinitialspace=True)

# Clean up column names by removing any leading/trailing commas
df.columns = df.columns.str.strip(',')

# Change movie titles from "Matrix, The" to "The Matrix" format
df['movie_name'] = df['movie_name'].str.replace(r'^(.*?),\s*(The|A|An)\s*$', r'\2 \1', regex=True)

# Select only the desired columns
filtered_df = df[[
    'movie_name',
    'reviewer_name',
    'review_text'
]]

# Save the filtered data to a new CSV
filtered_df.to_csv('data/filtered_reviews1.csv', index=False)

# Print the first few rows to verify the changes
print("\nFirst few rows of filtered data:")
print(filtered_df.head())


First few rows of filtered data:
   review_id       movie_name  reviewer_name  \
0          1  Lethal Weapon 3    J. Boyajian   
1          2  Lethal Weapon 3  Frank Maloney   
2          3  Lethal Weapon 3       Brian L.   
3          4  Lethal Weapon 3   Mark Santora   
4          5  Lethal Weapon 3        Jose R.   

                                         review_text rated  
0  About 20 minutes into LETHAL WEAPON 3, my neph...     R  
1  LETHAL WEAPON 3 is a film directed by Richard ...     R  
2  120 min., R, Comedy/Action, 1992\nDirector: Ri...     R  
3  It has been a couple of years since we last sa...     R  
4  I went to this movie with very low expectation...     R  


### Find and asssociate each tconst id to each review, based on a 80% accuracy test

In [ ]:
from fuzzywuzzy import fuzz
import pandas as pd
from tqdm import tqdm
import time

# Read the TSV file
movies_df = pd.read_csv('data/top_10000_weighted_ratings.tsv', sep='\t')

# Read the filtered reviews CSV
reviews_df = pd.read_csv('data/filtered_reviews1.csv')

# Function to find best matching movie title
def find_best_match(title, movie_titles_dict, threshold=80):
    best_match = None
    best_score = 0
    
    for movie_title, movie_id in movie_titles_dict.items():
        score = fuzz.ratio(title.lower(), movie_title.lower())
        if score > threshold and score > best_score:
            best_score = score
            best_match = movie_id
    
    return best_match

# Create dictionary of movie titles to tconst
title_to_tconst = dict(zip(movies_df['primaryTitle'], movies_df['tconst']))

# Start time measurement
start_time = time.time()

# Apply fuzzy matching to each movie title with progress bar
tqdm.pandas(desc="Matching movie titles")
reviews_df['tconst'] = reviews_df['movie_name'].progress_apply(
    lambda x: find_best_match(x, title_to_tconst)
)

# Calculate elapsed time
elapsed_time = time.time() - start_time

# Remove rows where no match was found
reviews_df = reviews_df.dropna(subset=['tconst'])

# Save the filtered data back to CSV
reviews_df.to_csv('data/filtered_reviews1.csv', index=False)

# Print statistics and timing information
print(f"\nProcessing completed in {elapsed_time:.2f} seconds")
print(f"Number of reviews after fuzzy matching: {len(reviews_df)}")
print(f"Number of unique movies in filtered reviews: {reviews_df['movie_name'].nunique()}")

# Print some sample matches to verify
sample = reviews_df[['movie_name', 'tconst']].head(10)
sample['matched_title'] = sample['tconst'].map(dict(zip(movies_df['tconst'], movies_df['primaryTitle'])))
print("\nSample matches:")
print(sample[['movie_name', 'matched_title']])

Matching movie titles: 100%|██████████| 18862/18862 [05:58<00:00, 52.68it/s]



Processing completed in 358.05 seconds
Number of reviews after fuzzy matching: 8991
Number of unique movies in filtered reviews: 1055

Sample matches:
        movie_name  matched_title
0  Lethal Weapon 3  Lethal Weapon
1  Lethal Weapon 3  Lethal Weapon
2  Lethal Weapon 3  Lethal Weapon
3  Lethal Weapon 3  Lethal Weapon
4  Lethal Weapon 3  Lethal Weapon
5     American Pie   American Pie
6     American Pie   American Pie
7     American Pie   American Pie
8     American Pie   American Pie
9     American Pie   American Pie


Number of reviews after fuzzy matching: 8991
Number of unique movies in filtered reviews: 1055

Number of reviews after filtering: 7826
Number of unique movies in filtered reviews: 846

Number of reviews after filtering: 6916
Number of unique movies in filtered reviews: 771

## REVIEWS 2

In [ ]:
# Read the CSV file with the correct separator
df = pd.read_csv('data/movies_reviews2.csv', sep=',', skipinitialspace=True)

# Clean up column names by removing any leading/trailing commas
df.columns = df.columns.str.strip(',')

# Clean movie titles by removing content within parentheses
df['title'] = df['title'].str.replace(r'\s*\([^)]*\)', '', regex=True)

# Select only the desired columns
filtered_df = df[[
    'title',
    'quote',
    'author'
]]

# Save the filtered data to a new CSV
filtered_df.to_csv('data/filtered_reviews2.csv', index=False)

# Print the first few rows to verify the changes
print("\nFirst few rows of filtered data:")
print(filtered_df.head())


First few rows of filtered data:
     title                                              quote        author
0  Dekalog  What a bitter pill to swallow when knowing the...          Siza
1  Dekalog  The best 10 hours you will spend at movies and...  ItsMahdiyarr
2  Dekalog  Touching and human, Dekalog is a trip inside m...      Andremax
3  Dekalog  Krzysztof Kieslowski's "The Decalogue" is one ...        thelol
4  Dekalog  For me it is simply breathtaking what a tv ser...      Daki1105


In [ ]:
from fuzzywuzzy import fuzz
import pandas as pd
from tqdm import tqdm
import time

# Read the TSV file
movies_df = pd.read_csv('data/top_10000_weighted_ratings.tsv', sep='\t')

# Read the filtered reviews CSV
reviews_df = pd.read_csv('data/filtered_reviews2.csv')

# Function to find best matching movie title
def find_best_match(title, movie_titles_dict, threshold=80):
    best_match = None
    best_score = 0
    
    for movie_title, movie_id in movie_titles_dict.items():
        score = fuzz.ratio(title.lower(), movie_title.lower())
        if score > threshold and score > best_score:
            best_score = score
            best_match = movie_id
    
    return best_match

# Create dictionary of movie titles to tconst
title_to_tconst = dict(zip(movies_df['primaryTitle'], movies_df['tconst']))

# Start time measurement
start_time = time.time()

# Apply fuzzy matching to each movie title with progress bar
tqdm.pandas(desc="Matching movie titles")
reviews_df['tconst'] = reviews_df['title'].progress_apply(
    lambda x: find_best_match(x, title_to_tconst)
)

# Calculate elapsed time
elapsed_time = time.time() - start_time

# Remove rows where no match was found
reviews_df = reviews_df.dropna(subset=['tconst'])

# Save the filtered data back to CSV
reviews_df.to_csv('data/filtered_reviews2.csv', index=False)

# Print statistics and timing information
print(f"\nProcessing completed in {elapsed_time:.2f} seconds")
print(f"Number of reviews after fuzzy matching: {len(reviews_df)}")
print(f"Number of unique movies in filtered reviews: {reviews_df['title'].nunique()}")

# Print some sample matches to verify
sample = reviews_df[['title', 'tconst']].head(10)
sample['matched_title'] = sample['tconst'].map(dict(zip(movies_df['tconst'], movies_df['primaryTitle'])))
print("\nSample matches:")
print(sample[['title', 'matched_title']])

Matching movie titles:  61%|██████    | 404027/667536 [2:21:29<1:32:17, 47.59it/s]


AttributeError: 'float' object has no attribute 'lower'

Matching movie titles:  61%|██████    | 404027/667536 [2:21:29<1:32:17, 47.59it/s]

In [ ]:
# Read the filtered reviews CSV
reviews_df = pd.read_csv('data/filtered_reviews2.csv')

# Count total reviews with tconst
reviews_with_tconst = reviews_df['tconst'].notna().sum()

# Count unique movies (based on tconst)
unique_movies = reviews_df['tconst'].nunique()

# Calculate percentage of matched reviews
total_reviews = len(reviews_df)
match_percentage = (reviews_with_tconst / total_reviews) * 100

print(f"\nMatching Statistics:")
print(f"Total reviews in dataset: {total_reviews:,}")
print(f"Reviews with matched tconst: {reviews_with_tconst:,}")
print(f"Unique movies found: {unique_movies:,}")
print(f"Match rate: {match_percentage:.2f}%")

KeyError: 'tconst'